Cобрать информацию о вакансиях на вводимую должность с сайта superjob.ru или hh.ru. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [91]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [92]:
def _parser_item_hh(item):

    vacancy_data = {}
    
   
    vacancy_name = item.find('a', {'class': 'serp-item__title'}) \
                        .getText() \
                        .replace(u'\xa0', u' ')
    
    vacancy_data['vacancy_name'] = vacancy_name
    
   
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
   
  
    vacancy_data['salary'] = salary
    
  
    vacancy_link = item.find('a', {'class': 'serp-item__title'})['href']
    
  
    vacancy_data['vacancy_link'] = vacancy_link 
    
    # site
    vacancy_data['site'] = 'hh.ru'
    
    return vacancy_data

In [93]:
def _parser_hh(vacancy):

    vacancy_data = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0'}
        
    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('span', {'class': 'pager-item-not-in-short-range'})[3].getText())
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'})\
                                    .find_all('div', {'class': 'serp-item'})
                
        for item in vacancy_items:
            vacancy_data.append(_parser_item_hh(item))
            
    return vacancy_data

In [94]:
def parser_vacancy(vacancy):
        
    vacancy_data = []
    vacancy_data.extend(_parser_hh(vacancy))
   
    
    df = pd.DataFrame(vacancy_data)

    return df

In [96]:
vacancy = 'c++'
df = parser_vacancy(vacancy)

In [97]:
df

,vacancy_name,salary,vacancy_link,site
0,Разработчик C++,"[от , , 175 000, , , , руб.]",https://hh.ru/vacancy/73891459?from=vacancy_se...,hh.ru
1,Junior / Middle Qt(C++) разработчик,"[25 000 – 35 000 , , руб.]",https://hh.ru/vacancy/74192710?from=vacancy_se...,hh.ru
2,Ведущий программист С++,"[150 000 – 350 000 , , руб.]",https://hh.ru/vacancy/74014250?from=vacancy_se...,hh.ru
3,Программист С#/С++,"[до , , 5 000, , , , USD]",https://hh.ru/vacancy/73463792?from=vacancy_se...,hh.ru
4,Программист С++ (Junior/Middle),None,https://hh.ru/vacancy/73184980?from=vacancy_se...,hh.ru
...,...,...,...,...
175,С++ Software Engineer,None,https://hh.ru/vacancy/71157061?from=vacancy_se...,hh.ru
176,Разработчик С++/Qt [Кибербезопасность],None,https://hh.ru/vacancy/73398668?from=vacancy_se...,hh.ru
177,Ведущий разработчик С/С++,None,https://hh.ru/vacancy/73306358?from=vacancy_se...,hh.ru
178,Инженер внешней инфраструктуры со знанием С++,None,https://hh.ru/vacancy/73705919?from=vacancy_se...,hh.ru
